# 4. Stitch outputs together
- Once you have fitted the weak, medium and dense diads, stitch them all together, and apply the Ne correction model

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import joblib
import os
from pickle import load
import pickle
import DiadFit as pf
pf.__version__

'0.0.59'

In [2]:
MasterFolder = r"P:\WORK-GENERAL\POSTDOC-UCB\BERKELEY-VIBE\Documents\Projects\Data\Hawaii_FI\Data\Raman\Leilani-2018"
DayFolder = os.path.join(MasterFolder, "Leilani_2018FI_Nov322")

meta_path=DayFolder +  '\Metadata'
spectra_path=DayFolder + '\Spectra'
filetype='headless_txt'

if not os.path.exists(MasterFolder + '\OUTPUT'):
    os.mkdir(MasterFolder +'\OUTPUT')


output_path=MasterFolder +'\OUTPUT'


## Load in specra, and combine

In [3]:
from os import path
if path.exists('Discarded_df.xlsx'):
    discard=pd.read_excel('Discarded_df.xlsx')
else:
    discard=None
if path.exists('Weak_Diads.xlsx'):
    grp1=pd.read_excel('Weak_Diads.xlsx')
else:
    grp1=None
if path.exists('Medium_Diads.xlsx'):
    grp2=pd.read_excel('Medium_Diads.xlsx')
else:
    grp2=None
if path.exists('Strong_Diads.xlsx'):
    grp3=pd.read_excel('Strong_Diads.xlsx')
else:
    grp3=None
df2=pd.concat([grp1, grp2, grp3], axis=0)
if discard is not None:
    discard_cols=discard[discard.columns.intersection(df2.columns)]
    df2=pd.concat([df2, discard_cols])

In [4]:
df2.head()

,Unnamed: 0,filename,Splitting,Split_err_abs,Split_err_quadrature,Diad1_Combofit_Cent,Diad1_cent_err,Diad1_Combofit_Height,Diad1_Voigt_Cent,Diad1_Voigt_Area,...,HB2_Sigma,C13_Cent,C13_Area,C13_Sigma,Diad2_Gauss_Cent,Diad2_Gauss_Area,Diad2_Gauss_Sigma,Diad1_Gauss_Cent,Diad1_Gauss_Area,Diad1_Gauss_Sigma
0,0,LL8-3b-FI#1_CRR_DiadFit,103.193538,0.007243,0.005164,1285.701427,0.004089,1334.834930,1285.701427,1802.642969,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,LL8-3b-FI#2_CRR_DiadFit,103.206588,0.006751,0.004796,1285.676577,0.003705,1270.402541,1285.676577,1724.798654,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,LL8-3b-FI#3_CRR_DiadFit,103.153215,0.011125,0.007948,1285.822167,0.006364,544.337761,1285.822167,661.714034,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,LL8-3b-FI#4_CRR_DiadFit,103.144436,0.010933,0.007835,1285.832849,0.006366,592.412970,1285.832849,722.903324,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,LL8-3b-FI#5_CRR_DiadFit,103.193527,0.009547,0.006838,1285.740532,0.005541,670.102955,1285.740532,885.873567,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## First, get the metadata for this day

In [5]:
# Put the common string you can in all your Ne lines here
ID_str='_'
file_ext_meta='txt' # Will only take files of this type 
diad_meta=pf.get_files(path=meta_path,
file_ext='txt', exclude_str=['N', 'Si', 'series','IMG','Cap','window','nodiad'],
 sort=True)
diad_meta

['LL8-3b-FI#1.txt',
 'LL8-3b-FI#2.txt',
 'LL8-3b-FI#3.txt',
 'LL8-3b-FI#4.txt',
 'LL8-3b-FI#5.txt',
 'LL8-601-FI#1.txt',
 'LL8-601-FI#2_12mW.txt',
 'LL8-601-FI#2_6mW.txt',
 'LL8-601-FI#3wmelt.txt',
 'LL8-FI236-FI#1.txt',
 'LL8-FI236-FI#2wmelt.txt',
 'LL8-FI37-#1.txt',
 'LL8-FI37-#2.txt',
 'LL8-FI37-#3.txt',
 'LL8-P2636-FI#1.txt',
 'LL8-P2636-FI#2.txt',
 'LL8-P2636-FI#3.txt',
 'LL8-P2636-FI#4.txt',
 'LL8-P3609-FI#1_12mW.txt',
 'LL8-P3609-FI#1_6mW.txt',
 'LL8-P3609-FI#2_12mW.txt',
 'LL8-P3609-FI#2_6mW.txt',
 'LL8-P3609-FI#3_12mW.txt',
 'LL8-P3609-FI#3_6mW.txt',
 'de (1).txt',
 'de (2).txt',
 'de.txt']

## Now get all the important information out of the metadata files

In [6]:
meta=pf.stitch_metadata_in_loop_witec(path=meta_path, 
Allfiles=diad_meta, prefix=False,
trupower=True)
meta['filename'].iloc[0]

100%|██████████| 27/27 [00:00<00:00, 27.62it/s]

Done


'de.txt'

## Then get a simple file name you can stitch with the spectra

In [7]:
file_m=pf.extracting_filenames_generic(names=meta['filename'],
    prefix=False, str_prefix=" ",
   file_type='.txt')
# Checks they are all unique
file_m[0]

good job, no duplicate file names


'de'

## Now get filename from spectra into same form as metadata

In [8]:
# Remove these to get the pure file name
file_s=pf.extracting_filenames_generic(
    prefix=False, str_prefix=" ",
    names=df2['filename'].reset_index(drop=True),
   file_type='.txt')

# Remove the cosmic rays, as doesnt have it in the metatdata
file_s = np.char.replace(file_s.astype(str), "_CRR_DiadFit", "")

good job, no duplicate file names


In [9]:
file_s

array(['LL8-3b-FI#1', 'LL8-3b-FI#2', 'LL8-3b-FI#3', 'LL8-3b-FI#4',
       'LL8-3b-FI#5', 'LL8-601-FI#2_12mW', 'LL8-601-FI#2_6mW',
       'LL8-601-FI#3wmelt', 'LL8-FI236-FI#1', 'LL8-FI236-FI#2wmelt',
       'LL8-FI37-#1', 'LL8-FI37-#2', 'LL8-FI37-#3', 'LL8-P2636-FI#1',
       'LL8-P2636-FI#2', 'LL8-P2636-FI#3', 'LL8-P2636-FI#4',
       'LL8-P3609-FI#3_6mW', 'LL8-601-FI#1', 'LL8-P3609-FI#1_12mW',
       'LL8-P3609-FI#1_6mW', 'LL8-P3609-FI#2_12mW', 'LL8-P3609-FI#2_6mW',
       'LL8-P3609-FI#3_12mW'], dtype='<U19')

In [10]:
# Combining them together
meta['name_for_matching']=file_m
df2['name_for_matching']=file_s
df2['Name_for_Secondary_Phases']=file_s
df_combo=df2.merge(meta, on='name_for_matching')

In [11]:
Ne_corr=pf.calculate_Ne_corr_std_err_values(pickle_str='polyfit_data.pkl', 
    new_x=df_combo['sec since midnight'], CI=0.67)
Ne_corr.head()


0


,time,preferred_values,lower_values,upper_values
0,42648.0,0.998012,0.998005,0.998019
1,43472.0,0.998006,0.997999,0.998013
2,44739.0,0.997997,0.997990,0.998004
3,45038.0,0.997995,0.997987,0.998002
4,45476.0,0.997991,0.997984,0.997998


In [12]:

split_err=pf.propagate_errors_for_splitting(Ne_corr, df_combo)
df_combo_out=df_combo.copy()
df_combo_out.insert(1, 'Corrected_Splitting', df_combo['Splitting']*Ne_corr['preferred_values'])
df_combo_out.insert(2, 'Corr_Split+1σ', df_combo_out['Corrected_Splitting']+split_err)
df_combo_out.insert(3, 'Corr_Split-1σ',df_combo_out['Corrected_Splitting']-split_err)
df_combo_out.insert(4, 'Corr_Split_1σ_val', split_err)

nm=os.path.basename(DayFolder)
df_combo_out.to_excel(output_path+'/'+nm+'_FI_fitting.xlsx')

## If you have secondary phases, now is the time to merge those in

In [13]:
if path.exists('Carb_Peak_fits.xlsx'):
    Carb=pd.read_excel('Carb_Peak_fits.xlsx')
else:
    Carb=None
if path.exists('SO2_Peak_fits.xlsx'):
    SO2=pd.read_excel('SO2_Peak_fits.xlsx')
else:
    SO2=None
if SO2 is not None and Carb is not None:
    Sec_Phases=pd.merge(SO2, Carb, on='filename', how='outer')
elif SO2 is not None and Carb is None:
    Sec_Phases=SO2
elif SO2 is None and Carb is not None:
    Sec_Phases=Carb
else:
    Sec_Phases=None
Sec_Phases.head()

,filename,Peak_Cent_SO2,Peak_Area_SO2,Peak_Height_SO2,Model_name
0,LL8-FI236-FI#1.txt,1150.740082,1172.611758,819.050408,Spline
1,LL8-P2636-FI#3.txt,1150.887569,226.862425,138.762685,Spline
2,LL8-P3609-FI#1_12mW.txt,1150.946564,19.063772,13.730796,Spline


In [14]:
# Remove these to get the pure file name
if Sec_Phases is not None:
    file_sec_phase=pf.extracting_filenames_generic(
        prefix=False, str_prefix=" ",
        names=Sec_Phases['filename'].reset_index(drop=True),
       file_type='.txt')

    file_sec_phase

good job, no duplicate file names


In [15]:
file_sec_phase

array(['LL8-FI236-FI#1', 'LL8-P2636-FI#3', 'LL8-P3609-FI#1_12mW'],
      dtype=object)

In [16]:
df_combo['name_for_matching']

0             LL8-3b-FI#1
1             LL8-3b-FI#2
2             LL8-3b-FI#3
3             LL8-3b-FI#4
4             LL8-3b-FI#5
5       LL8-601-FI#2_12mW
6        LL8-601-FI#2_6mW
7       LL8-601-FI#3wmelt
8          LL8-FI236-FI#1
9     LL8-FI236-FI#2wmelt
10            LL8-FI37-#1
11            LL8-FI37-#2
12            LL8-FI37-#3
13         LL8-P2636-FI#1
14         LL8-P2636-FI#2
15         LL8-P2636-FI#3
16         LL8-P2636-FI#4
17     LL8-P3609-FI#3_6mW
18           LL8-601-FI#1
19    LL8-P3609-FI#1_12mW
20     LL8-P3609-FI#1_6mW
21    LL8-P3609-FI#2_12mW
22     LL8-P3609-FI#2_6mW
23    LL8-P3609-FI#3_12mW
Name: name_for_matching, dtype: object

In [17]:
df_combo['name_for_matching']=df_combo['Name_for_Secondary_Phases']

if Sec_Phases is not None:
    Sec_Phases['name_for_matching']=file_sec_phase
    df_combo_sec_phase=df_combo_out.merge(Sec_Phases, 
    on='name_for_matching', how='outer')
    
else:
    df_combo_sec_phase=df_combo_out

### Add some more useful columns

In [18]:
if 'Peak_Area_Carb' in df_combo_sec_phase.columns:
    df_combo_sec_phase['Carb_Diad_Ratio']=(df_combo_sec_phase['Peak_Area_Carb']/(df_combo_sec_phase['Diad1_Voigt_Area']
                      +df_combo_sec_phase['Diad2_Voigt_Area']))
if 'Peak_Area_SO2' in df_combo_sec_phase.columns:
    df_combo_sec_phase['SO2_Diad_Ratio']=(df_combo_sec_phase['Peak_Area_SO2']/(df_combo_sec_phase['Diad1_Voigt_Area']
                      +df_combo_sec_phase['Diad2_Voigt_Area']))
    
df_combo_sec_phase.head()

,Unnamed: 0,Corrected_Splitting,Corr_Split+1σ,Corr_Split-1σ,Corr_Split_1σ_val,filename_x,Splitting,Split_err_abs,Split_err_quadrature,Diad1_Combofit_Cent,...,duration,24hr_time,sec since midnight,Spectral Center,filename,Peak_Cent_SO2,Peak_Area_SO2,Peak_Height_SO2,Model_name,SO2_Diad_Ratio
0,0,102.988380,102.993586,102.983174,0.005206,LL8-3b-FI#1_CRR_DiadFit,103.193538,0.007243,0.005164,1285.701427,...,"['0h', '3m', '47s']",11:50:48 AM\n,42648.0,1325.004,NaN,NaN,NaN,NaN,NaN,NaN
1,0,103.000784,103.005626,102.995943,0.004842,LL8-3b-FI#2_CRR_DiadFit,103.206588,0.006751,0.004796,1285.676577,...,"['0h', '3m', '47s']",12:04:32 PM\n,43472.0,1325.004,NaN,NaN,NaN,NaN,NaN,NaN
2,0,102.946566,102.954531,102.938601,0.007965,LL8-3b-FI#3_CRR_DiadFit,103.153215,0.011125,0.007948,1285.822167,...,"['0h', '3m', '47s']",12:25:39 PM\n,44739.0,1325.004,NaN,NaN,NaN,NaN,NaN,NaN
3,0,102.937580,102.945433,102.929728,0.007853,LL8-3b-FI#4_CRR_DiadFit,103.144436,0.010933,0.007835,1285.832849,...,"['0h', '3m', '46s']",12:30:38 PM\n,45038.0,1325.004,NaN,NaN,NaN,NaN,NaN,NaN
4,0,102.986244,102.993106,102.979382,0.006862,LL8-3b-FI#5_CRR_DiadFit,103.193527,0.009547,0.006838,1285.740532,...,"['0h', '3m', '47s']",12:37:56 PM\n,45476.0,1325.004,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
df_combo_sec_phase.to_excel(output_path+'/'+nm+'_FI_fitting_w_sec_phases.xlsx')

In [20]:
from winotify import Notification, audio

toast= Notification(app_id="VSCode",title="Notebook completed",
                    msg="Step4_Stitch_Outputs_Together is done!",
                    duration="short")
toast.set_audio(audio.Mail,loop=False)
toast.show()